In [1]:
import pandas as pd
import ccxt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from datetime import datetime

import sys
sys.path.append("..") 

from tp_config import *
from tp_utils.data_provider import read_prices, read_data, load_data_from_exchange, market_to_symbol
import json

In [3]:
delisted_assets = ['MOD', 'SBTC', 'BCX', 'EON', 'ADD', 'ATD', 'EOP', 'MEETONE', 'IQ', 'VTHO']
pd.set_option('precision', 9)

def get_balance():
    json = exchange.fetch_balance()
    df = pd.DataFrame(json['info']['balances'])
    return df


def create_market_buy_order(exchange, market, amount):
    symbol = market_to_symbol(market)
    order = exchange.create_market_buy_order(symbol, amount)
    trades = exchange.fetchMyTrades (symbol, limit = 1)
    price = trades[-1]['price']
    return price

def create_market_sell_order(exchange, market, amount):
    symbol = market_to_symbol(market)
    order = exchange.create_market_sell_order(symbol, amount)
    trades = exchange.fetchMyTrades (symbol, limit = 1)
    price = trades[-1]['price']
    return price
  
def weights_to_df(cleaned_weights):
    dfw = pd.DataFrame.from_dict([cleaned_weights]).transpose()
    dfw.columns = ['W']
    dfw = dfw[dfw['W'] > 0]
    return dfw

def final_sums(df, total):
    xx = round(df * total, -1)
    return xx[xx['W'] > 50]

def load_data_for_portfolio(tickers):
    res = []
    for ticker in tickers:
        print(ticker)
        df = read_data(ticker, '1h')
        df = df.drop_duplicates(subset=['T'])
        df.index = df['T']
        df_close = df[['C']]
        df_close.columns = [ticker]
        res.append(df_close)

    df1 = res[0]
    for df2 in res[1:]:
        #df1 = df1.join(df2, how='inner', on= 'T')
        df1 = pd.merge_asof(df1, df2, on= 'T', tolerance = 1)
    df1 = df1.dropna(axis = 0)
    df_prices = df1.set_index('T') 
    return df_prices

def calc_weights(lookback, opt_type, par):
    df_period = df_prices.iloc[-lookback:]

    avg_returns = expected_returns.mean_historical_return(df_period)
    cov_mat = risk_models.sample_cov(df_period)

    # get weights maximizing the Sharpe ratio
    ef = EfficientFrontier(avg_returns, cov_mat)

    if opt_type == 'max_sharpe':
        weights = ef.max_sharpe()
    elif opt_type == 'efficient_risk':
        weights = ef.efficient_risk(par) 
    #weights = ef.max_quadratic_utility(0.3)

    print(weights)

    ef.portfolio_performance(verbose=False)
    cleaned_weights = ef.clean_weights()
    dfw = weights_to_df(cleaned_weights)
    return dfw

In [5]:
kshao_api = "LuvsqHIVmpvQKKGnPQZ4QbPWY6FYPhdldEk81zmoWO6ef4jHYWBa4Dgh0IKEaezZ"
kshao_secret = "kLyVpmF4b74y3KGD57wMJR7SA9cQ0op7SJZR6GfluCuXpEwk1l5KnzCwvsDHfzW2"

exchange = ccxt.binance({'apiKey': kshao_api, 'secret': kshao_secret})

## Current balance

In [268]:
df = get_balance()
df = df[df['free'].astype(float) > 0]
df = df[~df['asset'].isin(delisted_assets)]

ticker = 'BTC/USDT'
price_btc = exchange.fetch_ticker(ticker)['last']

balance = 0
balance_btc = 0

for row in df.iterrows():
    asset = row[1]['asset']
    amount = float(row[1]['free'])
    if asset == 'BTC':
        balance = balance + amount * price_btc
        balance_btc = balance_btc + amount
    else:
        ticker = asset + '/BTC'
        price = exchange.fetch_ticker(ticker)['last']
        balance = balance + amount * price * price_btc 
        balance_btc = balance_btc + amount*price
print('BTC: ', balance_btc)
print('USDT:', round(balance, 2))
  
    
df

BTC:  0.11216794427112001
USDT: 4386.8


,asset,free,locked
0,BTC,0.09200582,0.00000000
4,BNB,2.15223359,0.00000000


### Sell All

In [266]:
df = get_balance()
df = df[df['free'].astype(float) > 0]
df = df[~df['asset'].isin(delisted_assets)]
for row in df.iterrows():
    asset = row[1]['asset']
    amount = row[1]['free']

    if asset in ['BTC', 'USDT', 'BNB']:
        continue

    market = asset + '-BTC' 
    try:
        price_sell = create_market_sell_order(exchange, market, amount)
        print(market, price_sell)
    except:
        pass

ADA-BTC 4.147e-05


In [251]:
market = 'ADA-BTC'
amount = 10

### Sell market

In [245]:
price_sell = create_market_sell_order(exchange, market, amount)
print(price_sell)

5.699e-05


### Buy market

In [252]:
price_buy = create_market_buy_order(exchange, market, amount)
print(price_buy)

4.147e-05


### 0. Data from exchange

In [190]:
pair = 'BTC'
index_assets = pd.read_csv('index_assets.csv')['asset'].tolist()
markets = [s + '-' + pair for s in index_assets]
start_timestamp = exchange.parse8601('2019-01-01 00:00:00')
timeframes = {'1h':60}
data_path = DATA_PATH_CRYPTO + '/' + pair 
load_data_from_exchange(exchange, markets, timeframes, start_timestamp, data_path)

1546300800000 2019-01-01 04:00:00
AAVE-BTC
1h
ADA-BTC
1h
AVAX-BTC
1h
BNB-BTC
1h
BCH-BTC
1h
CAKE-BTC
1h
CTSI-BTC
1h
DOGE-BTC
1h
DOT-BTC
1h
EOS-BTC
1h
ETC-BTC
1h
ETH-BTC
1h
FTT-BTC
1h
LINK-BTC
1h
LTC-BTC
1h
LUNA-BTC
1h
MATIC-BTC
1h
NEO-BTC
1h
RUNE-BTC
1h
SOL-BTC
1h
SXP-BTC
1h
THETA-BTC
1h
TRX-BTC
1h
UNI-BTC
1h
VET-BTC
1h
WBTC-BTC
1h
XMR-BTC
1h
XRP-BTC
1h
ZIL-BTC
1h
Done
Time taken = 0.007  hours


### 1. Set index assets

In [199]:
portfolio = 'kshao'
portfolio_file = portfolio + '.json'

with open(portfolio_file) as f:
    portfolio_data = json.load(f)

opt_method = portfolio_data['opt_method']
    
pair = 'USDT'
pair = 'BTC'

main_assets = ['ETH', 'ETC', 'BNB', 'SOL', 'FTT',  'LINK', 'MATIC']

#if portfolio_data['base'][0] == 'index':

#else:
assets = index_assets
assets = main_assets
    
markets = [m + '-' + pair for m in assets]
portfolio_data

{'opt_method': 'max_sharpe',
 'base': ['main'],
 'markets': {'SOL-BTC': 1.498214447715895, 'MATIC-BTC': 302.60276252288236}}

### 2. Load data

In [192]:
df_prices =  load_data_for_portfolio(markets)   
print(datetime.fromtimestamp(df_prices.index[-1]/1000))

ETH-BTC
ETC-BTC
BNB-BTC
SOL-BTC
FTT-BTC
LINK-BTC
MATIC-BTC
2021-05-19 13:00:00


### 3. Calc weights

In [193]:
lookback = 24*7

if opt_method == 'max_sharpe': 
    dfw = calc_weights(lookback, opt_method, 0)
else:
    dfw = calc_weights(lookback, opt_method, 0.4)
print(opt_method)    
dfw 

OrderedDict([('ETH-BTC', 0.0), ('ETC-BTC', 0.0), ('BNB-BTC', 0.0), ('SOL-BTC', 0.0908257612411391), ('FTT-BTC', 0.0), ('LINK-BTC', 0.0), ('MATIC-BTC', 0.909174238758861)])
max_sharpe


,W
SOL-BTC,0.09083
MATIC-BTC,0.90917


### 4. Sell portfolio

In [173]:
print(portfolio_data)

# Sell portfolio    
balance = 0.0
portfolio = portfolio_data['markets']
for market in portfolio.keys():
    amount = portfolio[market]
    price_sell = create_market_sell_order(exchange, market, amount) 
    print(market, price_sell)
#    ticker = key.replace('-','/')
#    price_sell = exchange.fetch_ticker(ticker)['last']
    balance = balance + amount * price_sell
print('Current balance:', balance)       

{'opt_method': 'max_sharpe', 'base': ['main'], 'markets': {'SOL-BTC': 1.7296091232129576, 'MATIC-BTC': 138.57245724572456}}
SOL-BTC 0.00126554
MATIC-BTC 5.579e-05
Current balance: 0.0099198469195299


In [194]:
balance = 0.02

### 5. Set new portfolio

In [195]:
# buy 
portfolio = dfw.to_dict()['W']
overall_balance = balance
for key in portfolio.keys():
#    price_buy =  create_market_buy_order(exchange, market, amount)
    ticker = key.replace('-','/')
    price_buy = exchange.fetch_ticker(ticker)['last']
    sum_for_asset = portfolio[key] * overall_balance
    balance = balance - min(sum_for_asset, balance)
    quant = sum_for_asset / price_buy
    portfolio[key] = quant
    if balance < 0:
        print('Balance < 0')    
portfolio

{'SOL-BTC': 1.498214447715895, 'MATIC-BTC': 302.60276252288236}

### 6. Buy portfolio

In [196]:
for market in portfolio.keys():
    amount = portfolio[market]
    print(market, amount)
    price_buy =  create_market_buy_order(exchange, market, amount)

portfolio_data['markets'] = portfolio    
with open(portfolio_file, 'w') as f:
    json.dump(portfolio_data, f)    

SOL-BTC 1.498214447715895
MATIC-BTC 302.60276252288236


### End